In [167]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import torch
import os
import importlib

from scipy.signal import butter, filtfilt

import data_processing as d
import lstm_model as m
import transformer_model as tm
import bilstm_model as bm
import extract_gaze_from_video_lstm as e
import facing_estimator as fe
# import visualize_robot_pov_plotly as v

importlib.reload(e)
importlib.reload(d)
importlib.reload(m)
importlib.reload(tm)
importlib.reload(bm)
importlib.reload(fe)
# importlib.reload(v)

video_path = "../../../face_and_pose/videos/jpl_interaction_part1_iyuv/c1.avi"
# video_path =  "../../../datasets/MINT-RVAE-Dataset-for-multimodal-intent-prediction-in-human-robot-interaction-main/data/Env3/video_clean_session_20250731_154719.avi"

# video_path = "./videos/c8.avi" # wave

In [168]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

2.7.1+cu126
12.6
90501


In [169]:
# extract face direction and yolo poses (gaze disabled for speed)
yolo_full = d.load_yolo(video_path)
angles = None
draw_out = None


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1035) /home/ben-linux/ece1508/aer1515/Project/aer1515-model-lstm/aer1515-model-lstm/lstm_pipeline/../../../face_and_pose/videos/jpl_interaction_part1_iyuv/c1.avi: 480x640 2 persons, 32.0ms
video 1/1 (frame 2/1035) /home/ben-linux/ece1508/aer1515/Project/aer1515-model-lstm/aer1515-model-lstm/lstm_pipeline/../../../face_and_pose/videos/jpl_interaction_part1_iyuv/c1.avi: 480x640 2 persons, 26.5ms
video 1/1 (frame 3/1035) /home/ben-linux

In [170]:
# returns a list of extracted trajectories. Each item is a different person
yolo_results_list, yolo_boxes_list, yolo_start_frames, yolo_frame_idx_list = d.match_trajectories_yolo(yolo_full)
gaze_list = [None for _ in yolo_results_list]  # skip gaze matching for speed

In [ ]:
# Design filter
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Apply filter
def lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order)
    y = filtfilt(b, a, data)  # zero-phase filtering
    return y

# eyeballed based on formula (w/2)/tan(fov/2) camera focal length from fov and sensor size
k = np.array([
    [ 200, 0, 160],
    [0, -200, 200],
    [0, 0, 1]
    ])

# Set processing sample rates
input_freq = 30   # original video capture rate (Hz)
target_freq = 30  # desired processing rate (Hz). set to 30 to keep original framerate
print(f"Processing: input_freq={input_freq}Hz, target_freq={target_freq}Hz")

# Ensure the notebook reloads the latest code changes
importlib.reload(d)
importlib.reload(fe)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"testing on {device}")

# model inference time
model_dir = "./models"

# Load BiLSTM Model (Better for temporal understanding)
# model_path = os.path.join(model_dir, "bilstm_model.pth")
# model = bm.BidirectionalLSTMIntentClassifier(
#     input_size=36,
#     hidden_size=128,
#     num_layers=2,
#     dropout=0.3,
#     output_size=1
# ).to(device)

# #load LSTM model
# model_path = os.path.join(model_dir, "model.pth")
# model = m.SimpleLSTM(input_size=36, hidden_size=128, 
#                       num_layers=3, dropout=0.3).to(device)

#load transformer"
model_path = os.path.join(model_dir, "transformer_model.pth")
model = tm.SimpleTransformerIntentClassifier(
    input_size=36,  # 36
    d_model=192,  # INCREASED from 128 for more capacity
    nhead=6,  # INCREASED from 4 (d_model must be divisible by nhead: 192/6=32)
    num_layers=3,  # INCREASED from 2 for deeper representation
    downsample_layers=0,  
    use_velocity_features=True,
    dropout=0.15  # INCREASED slightly for better regularization
).to(device)


if device == 'cpu' or True:
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(model_path))

model.eval()
print(f"Loaded BiLSTM model with {bm.count_parameters(model):,} parameters")

pred_out_list = []
indices_list = []

X_list = []  # Collect all sequences from all people

# Diagnostic: report how many trajectories YOLO matched
print(f"Found {len(yolo_results_list)} YOLO trajectories")
for ti, yr in enumerate(yolo_results_list):
    try:
        print(f" Traj {ti}: original_frames={len(yr)}, shape={yr.shape}")
    except Exception:
        print(f" Traj {ti}: could not read shape")

# sequence length is in samples (frames) at target_freq
seq_len = 60  # number of samples (frames) per sequence at target_freq
for yolo_results in yolo_results_list:
    # process yolo output into format compatible with model (downsample + facing + yaw append)
    yolo_proc_out, indices, yaw_arr, facing_arr = d.process_yolo_trajectory(yolo_results, k, input_freq=input_freq, target_freq=target_freq, smooth_facing=True)
    indices = indices[0]
    data_arr = yolo_proc_out[0]
    n_dim, n_frames = data_arr.shape

    vid_len = n_frames

    if vid_len < seq_len: # skip if too short
        print(f" Skipping trajectory: vid_len={vid_len} < seq_len={seq_len}")
        pred_out_list += [None]
        indices_list += [None]
        continue

    # create_sequences now returns (X, y) — use correct sample rate for time offsets
    X, y = d.create_sequences(yolo_proc_out, seq_len, stride=1, sample_rate=target_freq)
    X_list.extend(X)  # ADD ALL SEQUENCES TO SINGLE LIST (not append the array)

    # Quick batch-standardize experiment (replace with train-set stats later)
    X_arr = np.array(X)  # shape (N, seq_len, d)
    flat = X_arr.reshape(-1, X_arr.shape[2])  # (N*seq_len, d)
    mean_feat = flat.mean(axis=0)[None, :]  # (1, d)
    std_feat = flat.std(axis=0)[None, :] + 1e-8
    print("Norm stats (first 8 means):", np.round(mean_feat.flatten()[:8],4))
    print("Norm stats (first 8 stds):", np.round(std_feat.flatten()[:8],4))
    X_norm = (X_arr - mean_feat.reshape(1,1,-1)) / std_feat.reshape(1,1,-1)

    X_tensor = torch.tensor(X_norm, dtype=torch.float32).to(device)

    print(yolo_proc_out[0].shape)
    print(f"input size with batch: {X_tensor.shape}")

    # Diagnostic prints: input feature ranges and NaNs
    if X_tensor.numel() > 0 and X_tensor.shape[0] > 0:
        flat2 = X_tensor.detach().cpu().numpy().reshape(-1, X_tensor.shape[2])
        print("Input dim:", X_tensor.shape[2])
        print("Input overall stats: min, max, mean, std =>", np.min(flat2), np.max(flat2), np.mean(flat2), np.std(flat2))
        print("Per-dim mean (first 8):", np.round(np.mean(flat2, axis=0)[:8], 4))
        print("Per-dim std  (first 8):", np.round(np.std(flat2, axis=0)[:8], 4))
        print("Any NaNs in input?:", np.isnan(flat2).any())
        print("Sample first sequence (first 6 features):")
        print(np.round(X_tensor[0, :, :6].cpu().numpy(), 4))

    # Skip model inference if no sequence windows were generated (empty batch)
    if X_tensor.numel() == 0 or X_tensor.shape[0] == 0:
        print("No sequence windows for this trajectory — skipping model inference.")
        pred = np.array([])
    else:
        # get logits and convert to probabilities before mapping
        logits = model(X_tensor)
        logits_np = logits.cpu().detach().numpy().flatten()
        probs = torch.sigmoid(logits).cpu().detach().numpy().flatten()
        pred = probs
        print("Logits stats: min,max,mean,std =>", np.min(logits_np), np.max(logits_np), np.mean(logits_np), np.std(logits_np))
        print("Prob stats:  min,max,mean,std =>", np.min(probs), np.max(probs), np.mean(probs), np.std(probs))
    pred_out_list.append(pred)
    indices_list.append(indices)

Processing: input_freq=30Hz, target_freq=30Hz
testing on cuda
Loaded BiLSTM model with 349,313 parameters
Found 14 YOLO trajectories
 Traj 0: original_frames=120, shape=(120, 17, 3)
 Traj 1: original_frames=298, shape=(298, 17, 3)
 Traj 2: original_frames=1, shape=(1, 17, 3)
 Traj 3: original_frames=4, shape=(4, 17, 3)
 Traj 4: original_frames=641, shape=(641, 17, 3)
 Traj 5: original_frames=7, shape=(7, 17, 3)
 Traj 6: original_frames=1, shape=(1, 17, 3)
 Traj 7: original_frames=1, shape=(1, 17, 3)
 Traj 8: original_frames=6, shape=(6, 17, 3)
 Traj 9: original_frames=4, shape=(4, 17, 3)
 Traj 10: original_frames=19, shape=(19, 17, 3)
 Traj 11: original_frames=397, shape=(397, 17, 3)
 Traj 12: original_frames=5, shape=(5, 17, 3)
 Traj 13: original_frames=118, shape=(118, 17, 3)
Norm stats (first 8 means): [    -0.1026      0.4914     -0.0986       0.501     -0.0967      0.5011     -0.0849      0.4982]
Norm stats (first 8 stds): [     0.0475      0.0319      0.0479      0.0338        0.

In [172]:
# Analyze prediction distributions
import matplotlib.pyplot as plt

print("="*60)
print("PREDICTION ANALYSIS")
print("="*60)

for i, pred_out in enumerate(pred_out_list):
    if pred_out is not None:
        valid_preds = pred_out[pred_out >= 0]  # exclude padding (-1)
        if len(valid_preds) > 0:
            print(f"\nPerson {i}:")
            print(f"  Total frames: {len(valid_preds)}")
            print(f"  Mean prediction: {valid_preds.mean():.3f}")
            print(f"  Median: {np.median(valid_preds):.3f}")
            print(f"  Min: {valid_preds.min():.3f}, Max: {valid_preds.max():.3f}")
            print(f"  Frames > 0.3: {(valid_preds > 0.3).sum()} ({100*(valid_preds > 0.3).mean():.1f}%)")
            print(f"  Frames > 0.5: {(valid_preds > 0.5).sum()} ({100*(valid_preds > 0.5).mean():.1f}%)")
            print(f"  Frames > 0.7: {(valid_preds > 0.7).sum()} ({100*(valid_preds > 0.7).mean():.1f}%)")

# Plot histogram of all predictions
all_preds = []
for pred_out in pred_out_list:
    if pred_out is not None:
        all_preds.extend(pred_out[pred_out >= 0])

if len(all_preds) > 0:
    plt.figure(figsize=(10, 4))
    plt.hist(all_preds, bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(0.5, color='red', linestyle='--', label='Default threshold (0.5)')
    plt.axvline(0.3, color='orange', linestyle='--', label='Alternative threshold (0.3)')
    plt.xlabel('Prediction value')
    plt.ylabel('Count')
    plt.title('Distribution of model predictions')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
    
    print(f"\nOverall: {len(all_preds)} predictions, mean={np.mean(all_preds):.3f}, median={np.median(all_preds):.3f}")

PREDICTION ANALYSIS

Person 0:
  Total frames: 60
  Mean prediction: 0.194
  Median: 0.000
  Min: 0.000, Max: 0.990
  Frames > 0.3: 13 (21.7%)
  Frames > 0.5: 12 (20.0%)
  Frames > 0.7: 11 (18.3%)

Person 1:
  Total frames: 238
  Mean prediction: 0.499
  Median: 0.455
  Min: 0.000, Max: 1.000
  Frames > 0.3: 122 (51.3%)
  Frames > 0.5: 117 (49.2%)
  Frames > 0.7: 112 (47.1%)

Person 4:
  Total frames: 581
  Mean prediction: 0.708
  Median: 0.995
  Min: 0.001, Max: 1.000
  Frames > 0.3: 420 (72.3%)
  Frames > 0.5: 414 (71.3%)
  Frames > 0.7: 405 (69.7%)

Person 11:
  Total frames: 337
  Mean prediction: 0.472
  Median: 0.153
  Min: 0.000, Max: 1.000
  Frames > 0.3: 162 (48.1%)
  Frames > 0.5: 158 (46.9%)
  Frames > 0.7: 154 (45.7%)

Person 13:
  Total frames: 58
  Mean prediction: 0.413
  Median: 0.279
  Min: 0.000, Max: 0.979
  Frames > 0.3: 28 (48.3%)
  Frames > 0.5: 21 (36.2%)
  Frames > 0.7: 18 (31.0%)


<IPython.core.display.Javascript object>


Overall: 1274 predictions, mean=0.569, median=0.925


In [173]:
%matplotlib notebook

# Print info about sequence distribution
print(f"Total sequences: {len(X_list)}")
print(f"X shape: {X.shape}")

# Visualize sequences with gaps to see different people
indices_to_check = [0, 50, 100, 150, 200]
for idx in indices_to_check:
    if idx < len(X):
        print(f"\nVisualizing sequence {idx}")
        ani = d.plot_validate(X[idx])
        display(HTML(ani.to_jshtml()))

Total sequences: 1274
X shape: (58, 60, 36)

Visualizing sequence 0


<IPython.core.display.Javascript object>


Visualizing sequence 50


<IPython.core.display.Javascript object>

In [174]:
# pred_out_list_tweaked = pred_out_list.copy()
# pred_out_list_tweaked[0][pred_out_list_tweaked[0] > 0.5] = 0
# pred_out_list_tweaked[4][70:200] = 1
# pred_out_list_tweaked[4][550:] = 0

# plt.plot(range(0, len(pred_out_list_tweaked[4])), pred_out_list_tweaked[4])

In [175]:
# animations
# ani = d.draw_points_matplot(yolo_proc_out, use_array q= True, only_2d = True)
# display(HTML(ani.to_jshtml()))
# set start_frame=None to render from the beginning
d.draw_features(video_path, yolo_results_list, yolo_start_frames, gaze_list = gaze_list,
                 yolo_boxes_list = yolo_boxes_list, pred_results_list = pred_out_list, 
                fps_target = 30, start_frame = None, draw_gaze=True, frame_idx_list=yolo_frame_idx_list)

no frame returned
